In [8]:
import pandas as pd
import numpy as np
import random
import string

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_letters+string.digits
    rand_int = random.randint(1,4)
#     print(rand_int)
    string_random = [random.choice(letters) for i in range(stringLength)]
    chunk_array = []
    for chunk in chunks(string_random,rand_int):
        chunk_array.append("".join(chunk))
#     for i in range(rand_int+1):
    
#         string_random = string_random +" " + ''.join(random.choice(letters) for i in range(stringLength))
    
    return chunk_array

print(randomString())

['SWBF', 'HAnL', 'ht']


In [9]:
print(randomString())

['Q3', 'Vr', 'yX', 'rs', 'OS']


In [10]:
data = open("dummyData.txt").readlines()

In [11]:
import spacy

nlp = spacy.load("en_core_web_sm")


In [12]:
array = []
for textindex,text in enumerate(data):
    doc = nlp(text.strip())
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #        token.shape_, token.is_alpha, token.is_stop)
        text_items = [token.text]
        if token.text == "XYYZ":
            tag = "Destination"
            rand_int = random.randint(1,4)
            if rand_int==3:
                text_items = ["Unknown"]
            else:
                text_items = randomString()
        else:
            tag = ""
            rand_int = random.randint(1,10)
            if rand_int==3:
                text_items = ["Unknown"]
        for item in text_items:
            array.append(dict(Sentence=textindex,Word=item,POS=token.pos_,Tag=tag))

In [13]:
df = pd.DataFrame(array)  

In [14]:
df.tail(10)

,POS,Sentence,Tag,Word
1107,ADV,121,,home
1108,PRON,122,,I
1109,VERB,122,,got
1110,PART,122,,to
1111,AUX,122,,get
1112,NOUN,122,,directions
1113,ADP,122,,to
1114,PROPN,122,Destination,t1kI
1115,PROPN,122,Destination,jGza
1116,PROPN,122,Destination,Xn


In [15]:
words = list(set(df["Word"].values))
words.append("ENDPAD")
words.append("Unknown")
n_words = len(words); n_words

318

In [16]:
tags = list(set(df["Tag"].values))
n_tags = len(tags); n_tags

pos = list(set(df["POS"].values))
pos.append("BLANK")
pos_tags = len(pos)
print(n_tags,pos_tags)

2 13


In [17]:
agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                   s["POS"].values.tolist(),
                                                   s["Tag"].values.tolist())]
grouped = df.groupby("Sentence").apply(agg_func)

In [18]:
sentences = [s for s in grouped]

In [19]:
print(sentences[0])

[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('xeof', 'PROPN', 'Destination'), ('Vrhu', 'PROPN', 'Destination'), ('oq', 'PROPN', 'Destination')]


In [20]:
def yield_sentence():
    for group in grouped:
        yield group
    yield None

In [21]:
for group in yield_sentence():
    print(group)
    print("-------")

[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('xeof', 'PROPN', 'Destination'), ('Vrhu', 'PROPN', 'Destination'), ('oq', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Unknown', 'PROPN', 'Destination'), ('home', 'ADV', '')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Unknown', 'PROPN', 'Destination'), ('work', 'NOUN', '')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('the', 'DET', ''), ('closest', 'ADJ', ''), ('Unknown', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('the', 'DET', ''), ('nearest', 'ADJ', ''), ('Unknown', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'PART', ''), ('go', 'VERB', ''), ('home', 'ADV', '')]
-------
[('directions', 'NOUN', ''), ('to', 'PART', ''), ('Unknown', 'VERB', ''), ('to', 'ADP', ''), ('Unknown', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'PART', ''), ('go', 'VERB', ''), ('Unknown', 'ADP', ''), ('P', 'PROPN

In [22]:
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
pos2idx = {p: i for i, p in enumerate(pos)}


In [23]:
word2idx["directions"]

137

In [24]:
tag2idx[""]
pos2idx

{'PUNCT': 0,
 'NUM': 1,
 'PRON': 2,
 'VERB': 3,
 'ADV': 4,
 'NOUN': 5,
 'PROPN': 6,
 'ADP': 7,
 'PART': 8,
 'DET': 9,
 'AUX': 10,
 'ADJ': 11,
 'BLANK': 12}

In [25]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx.get(w[0],word2idx["Unknown"]) for w in s] for s in sentences]
X_pos = [[pos2idx.get(w[1]) for w in s] for s in sentences]

Using TensorFlow backend.


In [26]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)
X_pos = pad_sequences(maxlen=max_len, sequences=X_pos, padding="post", value=pos2idx["BLANK"])
# print(X.shape)
# print(X_pos.shape)

# X = np.stack((X,X_pos))
# print(X.shape)
# X = np.reshape(X,[123,2,75])
# print(X.shape)

In [27]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]

In [28]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx[""])


In [29]:
from keras.utils import to_categorical

In [30]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_tr, X_te, y_tr, y_te, X_pos_tr, X_pos_te = train_test_split(X, y, X_pos, test_size=0.1)

In [33]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
import keras

In [36]:
GLOVE_DIR = "glove.6B.50d.txt"
def load_glove():
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open(GLOVE_DIR, encoding="utf8")
    words = []
    for line in f:
        values = line.split()
        word = values[0]
        words.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    # create a weight matrix for words
    embedding_matrix = np.zeros((len(embeddings_index)+1, 50))
    for i in range(len(words)):
        embedding_vector = embeddings_index.get(words[i])
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


    print('Loaded %s word vectors.' % len(embedding_matrix))
    return words, embedding_matrix

words_list, embedding_matrix = load_glove()

Loaded 400001 word vectors.


In [38]:
# input = Input(shape=(max_len,))
# model = Embedding(input_dim=n_words + 1, output_dim=20,
#                   input_length=max_len)(input)  # 20-dim embedding
# model = Bidirectional(LSTM(units=50, return_sequences=True,
#                            recurrent_dropout=0.1))(model)  # variational biLSTM
# model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
# crf = CRF(n_tags)  # CRF layer
# out = crf(model)  # output


input = Input(shape=(75,))
glove_emb = Embedding(len(words_list) + 1, 50, weights=[embedding_matrix], trainable=False)(input)


# input = Input(shape=(max_len,))
# word_emb = Embedding(input_dim=n_words + 1, output_dim=20,
# input_length=max_len)(input) # 20-dim embedding

pos_input = Input(shape=(max_len,))
pos_emb = Embedding(input_dim= len(pos) , output_dim=10,
input_length=max_len)(pos_input)

modified_input = keras.layers.concatenate([glove_emb, pos_emb])

model_1 = Bidirectional(LSTM(units=50, return_sequences=True,
recurrent_dropout=0.1))(modified_input)

model = TimeDistributed(Dense(50, activation="relu"))(model_1)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output



In [39]:
model = Model([input,pos_input], out)

In [40]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

/home/premraj/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/home/premraj/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [41]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 75)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 75)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 75, 50)       20000050    input_4[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 75, 10)       130         input_5[0][0]                    
____________________________________________________________________________________________

In [42]:
print(X_tr[0].shape)

(75,)


In [43]:
history = model.fit([X_tr,X_pos_tr], np.array(y_tr), batch_size=32, epochs=60,
                    validation_split=0.1, verbose=1)

/home/premraj/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 99 samples, validate on 11 samples
Epoch 1/60
99/99 [==============================] - 2s 25ms/step - loss: 0.1423 - crf_viterbi_accuracy: 0.9541 - val_loss: 0.1015 - val_crf_viterbi_accuracy: 0.9479
Epoch 2/60
99/99 [==============================] - 0s 4ms/step - loss: 0.0935 - crf_viterbi_accuracy: 0.9582 - val_loss: 0.0780 - val_crf_viterbi_accuracy: 0.9479
Epoch 3/60
99/99 [==============================] - 0s 4ms/step - loss: 0.0779 - crf_viterbi_accuracy: 0.9493 - val_loss: 0.0755 - val_crf_viterbi_accuracy: 0.9479
Epoch 4/60
99/99 [==============================] - 0s 4ms/step - loss: 0.0722 - crf_viterbi_accuracy: 0.9577 - val_loss: 0.0671 - val_crf_viterbi_accuracy: 0.9673
Epoch 5/60
99/99 [==============================] - 0s 3ms/step - loss: 0.0640 - crf_viterbi_accuracy: 0.9662 - val_loss: 0.0818 - val_crf_viterbi_accuracy: 0.9491
Epoch 6/60
99/99 [==============================] - 0s 4ms/step - loss: 0.0637 - crf_viterbi_accuracy: 0.9717 - val_loss: 0.0595 - val_

99/99 [==============================] - 0s 4ms/step - loss: 0.0070 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0069 - val_crf_viterbi_accuracy: 1.0000
Epoch 51/60
99/99 [==============================] - 0s 3ms/step - loss: 0.0069 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0070 - val_crf_viterbi_accuracy: 1.0000
Epoch 52/60
99/99 [==============================] - 0s 3ms/step - loss: 0.0069 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0067 - val_crf_viterbi_accuracy: 1.0000
Epoch 53/60
99/99 [==============================] - 0s 3ms/step - loss: 0.0081 - crf_viterbi_accuracy: 0.9995 - val_loss: 0.0065 - val_crf_viterbi_accuracy: 1.0000
Epoch 54/60
99/99 [==============================] - 0s 4ms/step - loss: 0.0066 - crf_viterbi_accuracy: 0.9999 - val_loss: 0.0064 - val_crf_viterbi_accuracy: 1.0000
Epoch 55/60
99/99 [==============================] - 0s 4ms/step - loss: 0.0064 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0063 - val_crf_viterbi_accuracy: 1.0000
Epoch 56/60
99/99 [===

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report


In [116]:
test_pred = model.predict(X_te, verbose=1)

NameError: name 'X_te' is not defined

In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", ""))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [ ]:
print(test_labels)
print("-----")
print(pred_labels)
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

In [ ]:
sentence = "Take me to police station, quickly!"
entities_words = ["police","station"]
array = []
doc = nlp(sentence)
for token in doc:
    text = token.text
    print(text)
    if token.text in entities_words:
        tag = "Destination"
    else:
        tag = ""

    array.append(tuple([token.text,token.pos_,tag]))
print(array)
X = [[word2idx.get(w[0],word2idx["Unknown"]) for w in s] for s in [array]]
print(X)
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)
y = [[tag2idx[w[2]] for w in s] for s in [array]]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx[""])
y = [to_categorical(i, num_classes=n_tags) for i in y]
#print(y)
test_pred = model.predict(X, verbose=1)
pred_labels = pred2label(test_pred)
test_labels = pred2label(y)
print(test_labels,pred_labels)
#print(pred_labels)

In [ ]:
print(word2idx)